In [9]:
import random
import math
import pandas as pd
from datetime import datetime

In [10]:
def greedy_crate_selection(c_n, b_n, capacity):
    # Calculate the reward-to-weight ratio for each crate
    ratio = [(b_n[i] / c_n[i], c_n[i], b_n[i]) for i in range(len(c_n))]
    
    # Sort by the reward-to-weight ratio in descending order
    ratio.sort(reverse=True, key=lambda x: x[0])
    
    total_weight = 0
    total_reward = 0
    selected_crates = []
    
    for r, weight, reward in ratio:
        if total_weight + weight <= capacity:
            total_weight += weight
            total_reward += reward * weight
            # selected_crates.append((weight, reward, i))
            selected_crates.append(c_n.index(weight))
    
    return selected_crates, total_weight, total_reward


In [11]:
def highest_total_reward_heuristic(c_n, b_n, capacity):
    # Calculate the total reward for each crate
    total_rewards = [(c_n[i] * b_n[i], c_n[i], b_n[i]) for i in range(len(c_n))]
    
    # Sort by the total reward in descending order
    total_rewards.sort(reverse=True, key=lambda x: x[0])
    
    total_weight = 0
    total_reward = 0
    selected_crates = []
    
    for reward, weight, unit_reward in total_rewards:
        if total_weight + weight <= capacity:
            total_weight += weight
            total_reward += reward
            selected_crates.append(c_n.index(weight))
    
    return selected_crates, total_weight, total_reward

In [12]:
def lowest_weight_first_heuristic(c_n, b_n, capacity):
    crates = list(zip(c_n, b_n))
    
    crates.sort(key=lambda x: x[0])
    
    total_weight = 0
    total_reward = 0
    selected_crates = []
    
    for weight, unit_reward in crates:
        if total_weight + weight <= capacity:
            total_weight += weight
            total_reward += weight * unit_reward
            selected_crates.append(c_n.index(weight))
    
    return selected_crates, total_weight, total_reward

In [13]:
def calc_total_weight(c_n, state):
    return sum(c_n[i] * state[i] for i in range(len(c_n)))

def calc_total_reward(b_n, c_n, state):
    return sum(b_n[i] * c_n[i] * state[i] for i in range(len(b_n)))

def simulated_annealing(c_n, b_n, capacity):
    initial_temp=10000
    cooling_rate=0.99
    max_iterations= 1000000

    while True:
        init_state = [random.randint(0, 1) for _ in range(len(c_n))]
        if calc_total_weight(c_n, init_state) <= capacity:
            break

    current_state = init_state

    for i in range(max_iterations):
        # print(i)
        temp = initial_temp * math.pow(cooling_rate, i)
        if temp < 1e-6:
            break
        
        while True:
            next_state = current_state.copy()
            idx = random.randint(0, len(c_n) - 1)
            next_state[idx] = 1 - next_state[idx]

            if calc_total_weight(c_n, next_state) <= capacity:
                break

        current_reward = calc_total_reward(b_n, c_n, current_state)
        next_reward = calc_total_reward(b_n, c_n, next_state)

        if next_reward > current_reward or random.random() < math.exp((next_reward - current_reward) / temp):
            current_state = next_state
        
    total_weight = calc_total_weight(c_n, current_state)
    total_reward = calc_total_reward(b_n, c_n, current_state)
    selected_crates = [i for i in range(len(current_state)) if current_state[i] == 1]

    return selected_crates, total_weight, total_reward

In [14]:
c_n = [100, 155, 50, 112, 70, 80, 60, 118, 110, 55]
b_n = [1741, 1622, 1016, 1264, 1305, 1389, 1797, 1330, 1559, 1578]
capacity = 700


heuristics = [greedy_crate_selection, highest_total_reward_heuristic, lowest_weight_first_heuristic, simulated_annealing]

for heuristic in heuristics:
    selected_crates, total_weight, total_reward = heuristic(c_n, b_n, capacity)
    print("Heuristic:", heuristic.__name__) 
    print("Selected Crates (Weight, Index):", selected_crates)
    print("Total Weight:", total_weight)
    print("Total Reward:", total_reward)
    print("---------------------")

Heuristic: greedy_crate_selection
Selected Crates (Weight, Index): [6, 9, 2, 4, 0, 5, 8, 3]
Total Weight: 637
Total Reward: 935038
---------------------
Heuristic: highest_total_reward_heuristic
Selected Crates (Weight, Index): [1, 0, 8, 7, 3, 5]
Total Weight: 675
Total Reward: 1006628
---------------------
Heuristic: lowest_weight_first_heuristic
Selected Crates (Weight, Index): [2, 9, 6, 4, 5, 0, 8, 3]
Total Weight: 637
Total Reward: 935038
---------------------
Heuristic: simulated_annealing
Selected Crates (Weight, Index): [0, 1, 2, 3, 4, 5, 7]
Total Weight: 685
Total Reward: 977288
---------------------


In [15]:
excel_file = pd.ExcelFile('mochila_1.xlsx')
df = excel_file.parse('Hoja2')
df = df[[column for column in df if "Unn" not in column]]

df = df[(df['Contenedor'] != 'Contenedor')]
df = df.dropna()
df.rename(columns={'Contenedor': 'Variable', 'c5.1' : 'c6'}, inplace=True)

weights = df.loc[df['Variable'] == 'Peso', df.columns[1:]].reset_index(drop=True)
gain = df.loc[df['Variable'] == 'Beneficio [$/ton]', df.columns[1:]].reset_index(drop=True)

In [17]:
results = {}

for row in range(len(weights)):
    start_time = datetime.now()
    c_n = list(weights.loc[row])
    b_n = list(gain.loc[row])
    capacity = 700

    selected_crates, total_weight, total_reward = highest_total_reward_heuristic(c_n, b_n, capacity)
    delta_time = datetime.now() - start_time
    # print("Dataset: ", row + 1)
    # print("Heuristic:", highest_total_reward_heuristic.__name__) 
    # print("Selected Crates:", [element + 1 for element in selected_crates])
    # print("Total Weight:", total_weight)
    # print("Total Reward:", total_reward)
    # print("---------------------")

    results[row + 1] = {
        'Heuristic': highest_total_reward_heuristic.__name__,
        'Total Weight': total_weight,
        'Total Reward': total_reward,
        'Execution Time': delta_time
    }



pd.DataFrame(results).T

,Heuristic,Total Weight,Total Reward,Execution Time
1,highest_total_reward_heuristic,688,1098137,0:00:00
2,highest_total_reward_heuristic,640,1069306,0:00:00
3,highest_total_reward_heuristic,647,1108769,0:00:00
4,highest_total_reward_heuristic,645,1104584,0:00:00
5,highest_total_reward_heuristic,672,938679,0:00:00
6,highest_total_reward_heuristic,655,1033689,0:00:00
7,highest_total_reward_heuristic,670,953122,0:00:00
8,highest_total_reward_heuristic,699,1173518,0:00:00.002489
9,highest_total_reward_heuristic,648,970426,0:00:00
10,highest_total_reward_heuristic,667,1158197,0:00:00
